In [7]:
import os
import pandas as pd
from textblob import TextBlob
from dotenv import load_dotenv
import mysql.connector
import sqlalchemy
load_dotenv()
from transformers import pipeline

In [8]:
engine = sqlalchemy.create_engine(f'mysql+pymysql://{os.getenv("DB_USER")}:{os.getenv("DB_PASS")}@{os.getenv("DB_HOST")}/{os.getenv("DB_NAME")}')


In [9]:
df = pd.read_sql_table("comments", engine)
print(df)

     id  content_id                                       comment_text  \
0     1          13           none of the others looked like MrBeast 😂   
1     2          13                     this idea actually funny asf 😭   
2     3          13                           found him in 2 seconds 😂   
3     4          13                     mrbeast in parallel universes😂   
4     5          13   Only real ones can spot MrBeast’s brother, MrBro   
..  ...         ...                                                ...   
75  156          42  So adorable.  She&#39;s the cutest crazy cat l...   
76  157          42  The iconic duo is back!!!! Taylor and Olivia! ...   
77  158          42  Omgg!! Taylor hope you have the greatest time ...   
78  159          42   We missed you Tay Welcome back to the eras tour!   
79  160          42  Yes, Taylor! It&#39;s so sad that you&#39;re e...   

   sentiment_score  
0             None  
1             None  
2             None  
3             None  
4     

In [17]:

filtered_taylor_swift = df
# [df['content_id'] == '13']
# print(filtered_taylor_swift)

In [35]:
filtered_mrbeast = df[df['Name'] == 'MrBeast']


In [ ]:
filtered_taylor_swift['Polarity'] = filtered_taylor_swift['comment_text'].apply(lambda x: TextBlob(x).sentiment.polarity)
filtered_taylor_swift['Sentiment'] = filtered_taylor_swift['Polarity'].apply(
    lambda x: 'Positive' if x > 0 else 'Negative' if x < 0 else 'Neutral'
)
def assign_score(polarity):
    if polarity > 0:
        return round(5 + (5 * polarity))  # Scale from 6 to 10 for positive sentiments
    elif polarity < 0:
        return round(5 * polarity) + 5    # Scale from 1 to 4 for negative sentiments
    else:
        return 5  
    
# Step 5: Apply the scoring function
filtered_taylor_swift['Score'] = filtered_taylor_swift['Polarity'].apply(assign_score)

# Display the results with Comments, Sentiment, and Score
print(filtered_taylor_swift[['comment_text', 'Sentiment', 'Score']])

                                         comment_text Sentiment  Score
0            none of the others looked like MrBeast 😂   Neutral      5
1                      this idea actually funny asf 😭  Positive      6
2                            found him in 2 seconds 😂   Neutral      5
3                      mrbeast in parallel universes😂   Neutral      5
4    Only real ones can spot MrBeast’s brother, MrBro  Positive      6
..                                                ...       ...    ...
75  So adorable.  She&#39;s the cutest crazy cat l...  Negative      5
76  The iconic duo is back!!!! Taylor and Olivia! ...  Positive      6
77  Omgg!! Taylor hope you have the greatest time ...  Positive      7
78   We missed you Tay Welcome back to the eras tour!  Positive      7
79  Yes, Taylor! It&#39;s so sad that you&#39;re e...  Positive      7

[80 rows x 3 columns]
     id  content_id                                       comment_text  \
0     1          13           none of the others lo

In [27]:
import importlib.util

# Function to check if a package is installed
def check_package(package_name):
    return importlib.util.find_spec(package_name) is not None

# Check for TensorFlow and PyTorch
packages_installed = {
    'TensorFlow': check_package('tensorflow'),
    'PyTorch': check_package('torch')
}

print(packages_installed)

{'TensorFlow': True, 'PyTorch': True}


In [2]:
import os
os.environ['TF_USE_LEGACY_KERAS'] = '1'

In [3]:
import tensorflow as tf
from transformers import pipeline

# Step 1: Specify the exact model and revision
model_name = "distilbert-base-uncased-finetuned-sst-2-english"
revision = "714eb0f"  # Specific revision ID

# Step 2: Load the sentiment analysis pipeline with the specified model and revision
sentiment_pipeline = pipeline("sentiment-analysis", model=model_name, revision=revision)

# Step 3: Example comments related to Taylor Swift
comments = [
    "This song is amazing!",
    "I do not like this song.",
    "It was okay, nothing special."
]

# Step 4: Perform sentiment analysis on each comment
results = sentiment_pipeline(comments)

# Display results
for comment, result in zip(comments, results):
    print(f"Comment: {comment}")
    print(f"Sentiment: {result['label']}, Confidence: {result['score']:.2f}")

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


Comment: This song is amazing!
Sentiment: POSITIVE, Confidence: 1.00
Comment: I do not like this song.
Sentiment: NEGATIVE, Confidence: 0.99
Comment: It was okay, nothing special.
Sentiment: NEGATIVE, Confidence: 0.98


In [ ]:

sentiment_pipeline = pipeline("sentiment-analysis")

comments = [
    "This song is amazing!",
    "I do not like this song.",
    "It was okay, nothing special.",
    "This is one of the worst songs I have ever heard.",
    "Absolutely loved it! Can't stop listening.",
    "Meh, it's alright but not great.",
    "The lyrics are so touching and beautiful!",
    "This song makes me feel sad.",
    "I hate everything about this song.",
    "It's a masterpiece!"
]


results = sentiment_pipeline(comments)


def map_sentiment_to_score(sentiment_label, confidence):
    if sentiment_label == 'POSITIVE':
        # Map positive sentiments to scores between 6 and 10 based on confidence
        return round(5 + (5 * confidence))
    elif sentiment_label == 'NEGATIVE':
        # Map negative sentiments to scores between 1 and 4 based on confidence
        return round(5 * (1 - confidence))
    else:
        # Neutral sentiment gets a score of 5
        return 5

# Step 4: Assign scores based on the model's output
for comment, result in zip(comments, results):
    label = result['label']
    score = result['score']
    final_score = map_sentiment_to_score(label, score)
    
    print(f"Comment: {comment}")
    print(f"Sentiment: {label}, Confidence: {score:.2f}, Assigned Score: {final_score}")
    print("-" * 50)

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


Comment: This song is amazing!
Sentiment: POSITIVE, Confidence: 1.00, Assigned Score: 10
--------------------------------------------------
Comment: I do not like this song.
Sentiment: NEGATIVE, Confidence: 0.99, Assigned Score: 0
--------------------------------------------------
Comment: It was okay, nothing special.
Sentiment: NEGATIVE, Confidence: 0.98, Assigned Score: 0
--------------------------------------------------
Comment: This is one of the worst songs I have ever heard.
Sentiment: NEGATIVE, Confidence: 1.00, Assigned Score: 0
--------------------------------------------------
Comment: Absolutely loved it! Can't stop listening.
Sentiment: POSITIVE, Confidence: 1.00, Assigned Score: 10
--------------------------------------------------
Comment: Meh, it's alright but not great.
Sentiment: NEGATIVE, Confidence: 1.00, Assigned Score: 0
--------------------------------------------------
Comment: The lyrics are so touching and beautiful!
Sentiment: POSITIVE, Confidence: 1.00, Ass

In [12]:
from transformers import pipeline


sentiment_pipeline = pipeline("sentiment-analysis")


comments = df['comment_text'].tolist()


results = sentiment_pipeline(comments)


def map_sentiment_to_score(sentiment_label, confidence):
    if sentiment_label == 'POSITIVE':
        # Map positive sentiments to scores between 6 and 10 based on confidence
        return round(5 + (5 * confidence))
    elif sentiment_label == 'NEGATIVE':
        # Map negative sentiments to scores between 1 and 4 based on confidence
        return round(5 * (1 - confidence))
    else:
        # Neutral sentiment gets a score of 5
        return 5


for comment, result in zip(comments, results):
    label = result['label']
    score = result['score']
    final_score = map_sentiment_to_score(label, score)
    
    print(f"Comment: {comment}")
    print(f"Sentiment: {label}, Confidence: {score:.2f}, Assigned Score: {final_score}")
    print("-" * 50)

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


Comment: none of the others looked like MrBeast 😂
Sentiment: NEGATIVE, Confidence: 1.00, Assigned Score: 0
--------------------------------------------------
Comment: this idea actually funny asf 😭
Sentiment: POSITIVE, Confidence: 1.00, Assigned Score: 10
--------------------------------------------------
Comment: found him in 2 seconds 😂
Sentiment: NEGATIVE, Confidence: 0.96, Assigned Score: 0
--------------------------------------------------
Comment: mrbeast in parallel universes😂
Sentiment: NEGATIVE, Confidence: 0.99, Assigned Score: 0
--------------------------------------------------
Comment: Only real ones can spot MrBeast’s brother, MrBro
Sentiment: NEGATIVE, Confidence: 1.00, Assigned Score: 0
--------------------------------------------------
Comment: FUN.FACT mrbeast was it 12 cause mrbeast started YouTube channel in 2012
Sentiment: POSITIVE, Confidence: 1.00, Assigned Score: 10
--------------------------------------------------
Comment: Bro this was easy af
Sentiment: NEGAT